In [ ]:
import pandas as pd

filename1 = '/content/season_salarycap_data.csv'

cap_df = pd.read_csv(filename1, header=0)

# cap_df 데이터 전처리
cap_df.drop(['CONFIRMED', '% CHANGE', 'LOWER LIMIT '], axis=1, inplace=True)

for i in range(0, len(cap_df)):
  cap_df.loc[i, 'SEASON'] = cap_df.loc[i, 'SEASON'][:4]
  temp = []
  for n in cap_df.loc[i, 'UPPER LIMIT ']:
    if n.isdigit():
      temp.append(n)
  cap_df.loc[i, 'UPPER LIMIT '] = ''.join(temp)

  temp = []
  for n in cap_df.loc[i, 'MIN. SALARY ']:
    if n.isdigit():
      temp.append(n)
  cap_df.loc[i, 'MIN. SALARY '] = ''.join(temp)

for i in range(0, len(cap_df)):
  cap_df.loc[i, 'exchange rate'] = int(cap_df.loc[3, 'UPPER LIMIT ']) / int(cap_df.loc[i, 'UPPER LIMIT '])

cap_df.set_index('SEASON', inplace=True)

for year in range (2008, 2023):
  filename2 = '/content/' + str(year) + 'stat.csv'
  filename3 = '/content/' + str(year) + 'sal.csv'

  stat_df = pd.read_csv(filename2, header=0)
  sal_df = pd.read_csv(filename3, header=0)

  # stat_df 데이터 전처리
  stat_df.rename(columns={'Player':'PLAYER', 'PP':'PPG', 'PP.1':'PPA', 'SH':'SHG', 'SH.1': 'SHA'}, inplace=True)
  stat_df.drop(['Rk', 'Age', 'Tm', 'PIM', '+/-', 'FO%', '-9999'], axis=1, inplace=True)

  # sal_df 데이터 전처리
  for i in range(0, len(sal_df)):
    slice_name = sal_df.loc[i, 'PLAYER'].split(' ')
    sal_df.loc[i, 'PLAYER'] = slice_name[1] + ' ' + slice_name[2]    # Salary 데이터 Player 이름의 번호 제거
    cap  = []
    for n in sal_df.loc[i, 'CAP HIT']:
      if n.isdigit():
        cap.append(n)
    sal_df.loc[i, 'CAP HIT'] = ''.join(cap)

  sal_df.drop(['TEAM', 'AGE', 'DATE OF BIRTH', 'COUNTRY', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'CLAUSE'], axis=1, inplace=True)

  #데이터 병합
  merge_df = pd.merge(stat_df, sal_df, how='inner')

  for i in range(0, len(merge_df)):
    if int(merge_df.loc[i, 'CAP HIT']) - int(cap_df.loc[str(year), 'MIN. SALARY ']) < 0:
      merge_df.drop(i, inplace=True)
  merge_df.reset_index()
  for i in range(0, len(merge_df)):
    merge_df.loc[i, 'exchange cap'] = int((int(merge_df.loc[i, 'CAP HIT']) - int(cap_df.loc[str(year), 'MIN. SALARY '])) * cap_df.loc[str(year), 'exchange rate'])

  merge_df.to_csv(str(year) + "_merge.csv")

KeyError: ignored